# Web Scraping
# To do:
* Load booking.com and tripadvisor.com pages
* in search line insert city, date and number of people (It can be London, December 05-11th)
* search factors can differ according to page (tripadvisor might need room number)
* by default hotels are persented by popularity, so let's just take first 10, 15 or 20 and upload when to dataframe table
* table consist of: hotel name, price, rating, (can be number of reviews or smth else)


* After having two separate tables, maybe make one table if popular hotels are on both sites and when do comparison
* how many hotels are popular on both sites
* How prices differ on the sites (maybe make some graph)
* how ratings differ


In [78]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
import datetime

import pandas as pd
from bs4 import BeautifulSoup

In [126]:
browser=webdriver.Chrome()

In [127]:
#load Booking.com page
browser.get('https://www.booking.com')
assert 'Booking.com' in browser.title 


In [128]:
#Click accept button
accept = browser.find_element(By.ID, "onetrust-accept-btn-handler")
accept.click()

In [129]:
#Enter destination
search_box = browser.find_element(By.NAME, 'ss')
search_box.send_keys('London' + Keys.RETURN)

In [83]:
#If needed remove Genius advertisements
genius_element = browser.find_element(By.CSS_SELECTOR, '[aria-label="Dismiss sign in information."]')
genius_element.click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[aria-label="Dismiss sign in information."]"}
  (Session info: chrome=107.0.5304.110)
Stacktrace:
0   chromedriver                        0x000000010684f598 chromedriver + 4404632
1   chromedriver                        0x00000001067d6fa3 chromedriver + 3911587
2   chromedriver                        0x0000000106483e6a chromedriver + 425578
3   chromedriver                        0x00000001064bbc91 chromedriver + 654481
4   chromedriver                        0x00000001064bbe91 chromedriver + 654993
5   chromedriver                        0x00000001064f3274 chromedriver + 881268
6   chromedriver                        0x00000001064d9dcd chromedriver + 777677
7   chromedriver                        0x00000001064f0f79 chromedriver + 872313
8   chromedriver                        0x00000001064d9bd3 chromedriver + 777171
9   chromedriver                        0x00000001064aeced

In [132]:
#Press calendar button to enter check in date (without check out date system automatically sets one night stay)
calendar_button = browser.find_element(By.CLASS_NAME, 'd47738b911')
calendar_button.click()


In [134]:
#Move forward/backwards to choose the month of the stay
forward_calendar_page = browser.find_element(By.CLASS_NAME,"be298b15fa")
forward_calendar_page.click()

In [125]:
backward_calendar_page = browser.find_element(By.CLASS_NAME,"ab15620a33")
backward_calendar_page.click()


In [135]:
#Enter check in date
check_in_date = browser.find_element(By.CSS_SELECTOR, '[data-date="2023-02-09"]')
check_in_date.click()


In [136]:
submit = browser.find_element(By.CSS_SELECTOR, '[type=submit]')
submit.click()

In [147]:
#To avoid new accomodations let's set required review scores
number_list=list(range(90, 50, -10))
number_list
for i in number_list:
    review_score = browser.find_element(By.CSS_SELECTOR, f"[data-filters-item='review_score:review_score={i}']")
    review_score.click()


In [148]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [149]:
hotels = []
for name in soup.findAll('div',{'data-testid':'title'}):
  hotels.append(name.text.strip())
hotels[:5]
len(hotels)

25

In [150]:
ratings = []
for rating in soup.findAll('div',{'class':'d10a6220b4'}):
      ratings.append(rating.text.strip())
len(ratings)



25

In [151]:
reviews = []
for review in soup.findAll('div',{'class':'db63693c62'}):
  reviews.append(review.text.replace('reviews','').replace(',','').strip())
reviews[:5]
len(reviews)

25

In [152]:
price = []
for p in soup.findAll('span',{'class':['fbd1d3018c', 'bd73d13072']}):
  price.append(p.text.replace('€','').replace(',','').strip()) 
price[:5]
len(price)

25

In [153]:
d1 = {'Hotel':hotels,'Ratings':ratings,'No_of_Reviews':reviews,'Price, €':price}
df = pd.DataFrame.from_dict(d1)

In [154]:
df

,Hotel,Ratings,No_of_Reviews,"Price, €"
0,Langham Court Hotel,7.1,2282,180
1,Leonardo Royal London St Paul’s,8.2,2209,288
2,"Radisson Blu Edwardian Kenilworth Hotel, London",8.0,2251,268
3,The Dilly,7.4,2163,257
4,"The Westminster London, Curio Collection by Hi...",8.2,3001,168
5,Park Grand London Kensington,8.0,6024,259
6,Zedwell Piccadilly Circus,7.6,15024,119
7,Alexandra Hotel,7.0,3055,80
8,"Holiday Inn London Camden Lock, an IHG Hotel",8.2,3447,197
9,Studios2Let,7.7,5478,126
